In [ ]:
# First I install Apache Spark version 3.5.1 and set up the necessary environment

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [824 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,035 kB]
Get:13 http://security.ubuntu.co

In [2]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
import time
import datetime
from pyspark.sql.functions import year, to_date, regexp_replace

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

# Show DataFrame
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
# revise original column data formats:
# id remains unrevised
# I will extract the year from the 'date' column and create a new column called 'year_sold'
df = df.withColumn("year_sold", year(df['date']))
# I will extract the year from 'date_built' and rename it with same column name
df = df.withColumn("date_built", year(df['date_built']))
# I will convert columns below to Integer Type
df = df.withColumn("price", df["price"].cast(IntegerType()))
df = df.withColumn("bedrooms", df["bedrooms"].cast(IntegerType()))
df = df.withColumn("bathrooms", df["bathrooms"].cast(IntegerType()))
df = df.withColumn("sqft_living", df["sqft_living"].cast(IntegerType()))
df = df.withColumn("sqft_lot", df["sqft_lot"].cast(IntegerType()))
df = df.withColumn("floors", df["floors"].cast(IntegerType()))
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+---------+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year_sold|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+---------+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|     2022|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|     2021|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|     2019|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|     2019|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|     2022|


In [5]:
# 2. Create a temporary view of the DataFrame.
# I will create a temporary view of DataFrame 'df' called 'home_sales'
# This will allow me to perform Spark SQL operations on the DataFrame using SQL queries.
df.createOrReplaceTempView('home_sales')
# I will also print the schema of the DataFrame to provide an overview of the DataFrame's structure.
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)
 |-- year_sold: integer (nullable = true)



In [6]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("""
select year_sold, round(AVG(price), 2) as avgPrice
from home_sales
where bedrooms = 4
group by year_sold
order by year_sold DESC
"""
).show()

+---------+---------+
|year_sold| avgPrice|
+---------+---------+
|     2022|296363.88|
|     2021|301819.44|
|     2020|298353.78|
|     2019| 300263.7|
+---------+---------+



In [7]:
# 4. What is the average price of a home for each date the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""
select date_built, round(AVG(price),2) as avgPrice
from home_sales
where bedrooms = 3 and bathrooms= 3
group by date_built
order by date_built DESC
"""
).show()


+----------+---------+
|date_built| avgPrice|
+----------+---------+
|      2017|292676.79|
|      2016|290555.07|
|      2015| 288770.3|
|      2014|290852.27|
|      2013|295962.27|
|      2012|293683.19|
|      2011|291117.47|
|      2010|292859.62|
+----------+---------+



In [8]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("""
select date_built as year_built, round(AVG(price), 2) as avgPrice
from home_sales
where bedrooms = 3 and bathrooms = 3 and floors = 2 and sqft_living >= 2000
group by date_built
order by date_built DESC
"""
).show()


+----------+---------+
|year_built| avgPrice|
+----------+---------+
|      2017|280317.58|
|      2016| 293965.1|
|      2015|297609.97|
|      2014|298264.72|
|      2013|303676.79|
|      2012|307539.97|
|      2011|276553.81|
|      2010|285010.22|
+----------+---------+



In [9]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
select view, round(AVG(price), 2) as avgPrice
from home_sales
group by view
having avgPrice >= 350000
order by view DESC
"""
).show(100)

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view|  avgPrice|
+----+----------+
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
|  80| 991767.38|
|  79|1009565.08|
|  78|1080649.37|
|  77|1076205.56|
|  76|1058802.78|
|  75|1114042.94|
|  74|  745077.0|
|  73| 752861.18|
|  72| 780914.67|
|  71|  775651.1|
|  70| 695865.58|
|  69| 750537.94|
|  68| 716785.44|
|  67| 737970.96|
|  66|  712475.0|
|  65| 736679.93|
|  64| 767036.67|
|  63| 711614.55|
|  62| 759150.14|
|  61| 746877.59|
|  60| 754939.65|
|  59|  791453.0|
|  58| 759764.65|
|  57|  734340.5|
|  56|  718176.4|
|  55| 771153.32|
|  54| 798684.82|
|  53|  755214.8|
|  52| 733780.26|
|  51| 788128.21|
| 100| 1026669.5|
+----+----------+

--- 1.248741626739502 secon

In [10]:
# 7. Cache the the temporary table home_sales.
spark.sql('cache table home_sales')

DataFrame[]

In [11]:
# 8. Check if the table is cached.
is_cached = spark.catalog.isCached('home_sales')
print("Is home_sales table cached:", is_cached)

Is home_sales table cached: True


In [12]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""
select view, round(AVG(price),2) as avgPrice
from home_sales
group by view
having avgPrice >= 350000
order by view DESC
"""
).show(100)


print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view|  avgPrice|
+----+----------+
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
|  80| 991767.38|
|  79|1009565.08|
|  78|1080649.37|
|  77|1076205.56|
|  76|1058802.78|
|  75|1114042.94|
|  74|  745077.0|
|  73| 752861.18|
|  72| 780914.67|
|  71|  775651.1|
|  70| 695865.58|
|  69| 750537.94|
|  68| 716785.44|
|  67| 737970.96|
|  66|  712475.0|
|  65| 736679.93|
|  64| 767036.67|
|  63| 711614.55|
|  62| 759150.14|
|  61| 746877.59|
|  60| 754939.65|
|  59|  791453.0|
|  58| 759764.65|
|  57|  734340.5|
|  56|  718176.4|
|  55| 771153.32|
|  54| 798684.82|
|  53|  755214.8|
|  52| 733780.26|
|  51| 788128.21|
| 100| 1026669.5|
+----+----------+

--- 0.6661090850830078 seco

In [13]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy('date_built').parquet('home_sales',mode='overwrite')

In [14]:
# 11. Read the parquet formatted data.
p_df=spark.read.parquet('home_sales')

In [15]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('p_home_sales')

In [16]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""
select view, round(AVG(price), 2) as avgPrice
from p_home_sales
group by view
having avgPrice >= 350000
order by view DESC
"""
).show(100)

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view|  avgPrice|
+----+----------+
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
|  80| 991767.38|
|  79|1009565.08|
|  78|1080649.37|
|  77|1076205.56|
|  76|1058802.78|
|  75|1114042.94|
|  74|  745077.0|
|  73| 752861.18|
|  72| 780914.67|
|  71|  775651.1|
|  70| 695865.58|
|  69| 750537.94|
|  68| 716785.44|
|  67| 737970.96|
|  66|  712475.0|
|  65| 736679.93|
|  64| 767036.67|
|  63| 711614.55|
|  62| 759150.14|
|  61| 746877.59|
|  60| 754939.65|
|  59|  791453.0|
|  58| 759764.65|
|  57|  734340.5|
|  56|  718176.4|
|  55| 771153.32|
|  54| 798684.82|
|  53|  755214.8|
|  52| 733780.26|
|  51| 788128.21|
| 100| 1026669.5|
+----+----------+

--- 1.2266364097595215 seco

In [17]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [18]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached("home_sales")
print("Is home_sales table cached:", is_cached)

Is home_sales table cached: False
